# Dependencias

In [1]:
import geopandas as gpd
import folium
from shapely.geometry import Point, LineString, Polygon

# Tipo de datos

Geometrías básicas para la representación de datos geoespaciales (Point, LineString and Polygon)

In [3]:
# Crear las geometrías
punto = Point(-99.133209, 19.432608)  # Coordenadas de la Ciudad de México
linea = LineString([(-99.133209, 19.432608), (-100.366667, 20.666667)])  # De Ciudad de México a San Luis Potosí
poligono = Polygon([(-99.133209, 19.432608), (-99.133209, 19.40), (-99.10, 19.40), (-99.10, 19.432608)])  # Un polígono arbitrario en la Ciudad de México

# Crear un GeoDataFrame
gdf = gpd.GeoDataFrame({'geometry': [punto, linea, poligono]})

# Crear un mapa base
m = folium.Map(location=[19.432608, -99.133209], zoom_start=5)

# Añadir las geometrías al mapa
for _, row in gdf.iterrows():
    # Si la geometría es un punto, se añade un Marker
    if isinstance(row['geometry'], Point):
        folium.Marker(
            location=[row['geometry'].y, row['geometry'].x],
            popup='Punto'
        ).add_to(m)
    # Si la geometría es una línea, se añade un PolyLine
    elif isinstance(row['geometry'], LineString):
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in row['geometry'].coords],
            color='blue',
            weight=5,
            popup='Línea'
        ).add_to(m)
    # Si la geometría es un polígono, se añade un Polygon
    elif isinstance(row['geometry'], Polygon):
        folium.Polygon(
            locations=[(coord[1], coord[0]) for coord in row['geometry'].exterior.coords],
            color='green',
            weight=2,
            fill=True,
            fill_color='green',
            fill_opacity=0.2,
            popup='Polígono'
        ).add_to(m)

# Mostrar el mapa
m.save('mapa.html')  # Guardar el mapa en un archivo HTML
m  # Si estás en un entorno Jupyter, esto mostrará el mapa en la salida de la celda
